In [1]:
import nltk

nltk.download([
    "names",
    "stopwords",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
    ])

[nltk_data] Downloading package names to /Users/eltontay/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eltontay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/eltontay/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/eltontay/nltk_data...
[nltk_data] Downloading package punkt to /Users/eltontay/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np

from newspaper import Article
from newspaper import Config

import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

import string
import re

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df_world= pd.read_csv('../Data/Global/world_news.csv')
df_politics= pd.read_csv('../Data/Global/politics_news.csv.csv')
df_coronavirus= pd.read_csv('../Data/Global/coronavirus_news.csv')
df_aapl= pd.read_csv('../Data/Stock/aapl_news.csv')
df_meta= pd.read_csv('../Data/Stock/meta_news.csv')
df_tsla= pd.read_csv('../Data/Stock/tsla_news.csv')
print('world shape ', df_world.shape)
print('politics shape ',df_politics.shape)
print('coronavirus shape ',df_coronavirus.shape)
print('aapl shape ',df_aapl.shape)
print('meta shape ', df_meta.shape)
print('tsla shape ', df_tsla.shape)
print(df_world.head())

,Unnamed: 0,news_title,link,published
0,3270,Fed Plans to Raise Rates in 2018 but Lacks Con...,https://www.nytimes.com/2018/01/03/business/ec...,2018-01-03 08:00:00+00:00
1,3271,Forget Jay Powell: Thomas Jordan For Fed Chair...,https://dealbreaker.com/2018/01/forget-jay-pow...,2018-01-11 08:00:00+00:00
2,3272,Mester advocates for Fed to use rules-based ap...,https://www.cnbc.com/2018/01/17/mester-advocat...,2018-01-17 08:00:00+00:00
3,3275,Jerome Powell's Challenge at the Fed: Keep the...,https://www.wsj.com/articles/jerome-powells-ch...,2018-01-23 08:00:00+00:00
4,3274,Senate Confirms Jerome Powell as Federal Reser...,https://www.wsj.com/articles/senate-confirms-j...,2018-01-23 08:00:00+00:00


In [ ]:
df_world = df_world.drop(df_world.columns[0], axis=1)
df_politics = df_politics.drop(df_politics.columns[0], axis=1)
df_coronavirus = df_coronavirus.drop(df_coronavirus.columns[0], axis=1)
df_aapl = df_aapl.drop(df_aapl.columns[0], axis=1)
df_meta = df_meta.drop(df_meta.columns[0], axis=1)
df_tsla = df_tsla.drop(df_tsla.columns[0], axis=1)

In [ ]:
def remove_irrelevant_content(text):
    headline_only_string = "This headline-only article is meant to show you why a stock is moving, the most difficult aspect of stock trading"

    if headline_only_string in text:
        return ""
    else:
        return text

def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

def lowercase(text):
    return text.lower()

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

def remove_special_character(text):
    text = text.replace('\n', ' ') 
    return text

def lemmatize(text):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in text]
    return corpus

def preprocess_text(text):
    text = remove_irrelevant_content(text)
    text = remove_punctuation(text)
    text = lowercase(text)
    #text = remove_stopwords(text)
    text = remove_special_character(text)
    #text = lemmatize(text)
    return text

In [ ]:
sia = SentimentIntensityAnalyzer()

sia.polarity_scores(df_world['text'][0])

{'neg': 0.033, 'neu': 0.873, 'pos': 0.094, 'compound': 0.9245}

In [ ]:
df['sentiment score'] = df['processed content'].apply(lambda x: sia.polarity_scores(x))
df['positive'] = df['sentiment score'].apply(lambda x: x["compound"] > 0)

df.head()

,news_title,link,published,article content,processed content,sentiment score,positive
0,Fed Plans to Raise Rates in 2018 but Lacks Con...,https://www.nytimes.com/2018/01/03/business/ec...,2018-01-03 08:00:00+00:00,"In the December round of economic projections,...","In the December round of economic projections,...","{'neg': 0.033, 'neu': 0.873, 'pos': 0.094, 'co...",True
1,Forget Jay Powell: Thomas Jordan For Fed Chair...,https://dealbreaker.com/2018/01/forget-jay-pow...,2018-01-11 08:00:00+00:00,"Sure, the Federal Reserve made $80 billion las...","Sure, the Federal Reserve made $80 billion las...","{'neg': 0.028, 'neu': 0.797, 'pos': 0.175, 'co...",True
2,Mester advocates for Fed to use rules-based ap...,https://www.cnbc.com/2018/01/17/mester-advocat...,2018-01-17 08:00:00+00:00,The Federal Reserve should follow rules when c...,The Federal Reserve should follow rules when c...,"{'neg': 0.017, 'neu': 0.911, 'pos': 0.072, 'co...",True
5,"Senate confirms Powell for Fed chair, handing ...",https://www.washingtonpost.com/news/wonk/wp/20...,2018-01-23 08:00:00+00:00,Gift Article Share\n\nThe Senate overwhelmingl...,Gift Article Share The Senate overwhelmingly ...,"{'neg': 0.058, 'neu': 0.817, 'pos': 0.125, 'co...",True
6,"Meet Jerome Powell, One of the Most Powerful U...",https://fortune.com/2018/01/24/who-is-jerome-p...,2018-01-24 08:00:00+00:00,"With relatively little opposition, the U.S. Se...","With relatively little opposition, the U.S. Se...","{'neg': 0.048, 'neu': 0.912, 'pos': 0.04, 'com...",False


In [ ]:
df.to_csv('/content/drive/MyDrive/BT4222 Project/news articles/JeromePowell_VADER.csv')